# **Step 07 - The order tool**

We'll now add the missing part: **The order management.**

**Note**  
In order to make you concentrate on the new parts, all the code you have seen in the previous notebook has been grouped in the cell below, nothing new there, so just run it and follow what's next because is where things get more interesting for the task we want to achieve.


In [11]:
#region This cell contains most of the code you saw in the preceeding notebook. You can just run this cell and skip to the next one.

import os
import asyncio
import pytz
import json
from dotenv import load_dotenv

# Imports
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from datetime import datetime
from services.data_provider import DataProvider
from services.data_types import Order, OrderResponse
from termcolor import cprint
from langchain.agents import AgentExecutor, create_openai_functions_agent
from typing import Optional

# Environment setup
load_dotenv()

api_key=os.environ['AZURE_OPENAI_API_KEY']
endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
api_version=os.environ['AZURE_OPENAI_API_VERSION']
local_timezone = os.environ['TIMEZONE']

llm=AzureChatOpenAI(api_key=api_key, 
                    azure_endpoint=endpoint,
                    azure_deployment=deployment,
                    api_version=api_version, 
                    temperature=0.3)

# This the data access layer
db:DataProvider=DataProvider()

# Define the input schema for the schedule-tool
class ScheduleToolInputSchema(BaseModel):
    date: str = Field(description="The date for which you want to know the schedule in format 'DD/MM/YYYY HH:MM' (e.g. '31/12/2023 15:00').")
    user_id: Optional[str] = Field(description="The optional user id in form '#<4-digits-number>'.")
    
# Define the input schema for daily menu tool
class DailyMenuInputSchema(BaseModel):
    date: str = Field(description="The date for which you want to know the available menu in format 'DD/MM/YYYY HH:MM' (e.g. '29/05/2024 17:00').")    

# Returns the operative schedule for the provided date and the optional user id
@tool("schedule-tool", args_schema=ScheduleToolInputSchema)
def operative_schedule(date:str, user_id:str=None):
    """Useful when you need to know the operative schedule on a specific date and time."""
    weekday= datetime.strptime(date, "%d/%m/%Y %H:%M").strftime("%A")    
    
    # Check if the user is a special client
    is_special = db.is_special_client(user_id)
    # Get the operative schedule from the database
    schedule=db.get_schedule(day=weekday, is_special=is_special) 
    # Check if the requested date/timw comes within the working schedule       
    if schedule.status == "open":        
        time= datetime.strptime(date, "%d/%m/%Y %H:%M").time()
        opening = datetime.strptime(schedule.start, "%H:%M").time()
        closing = datetime.strptime(schedule.end, "%H:%M").time()        
        if time < opening or time > closing:
            schedule.status = "closed"            
        
    return schedule

# Returns the menu for the provided date
@tool("daily-menu-tool", args_schema=DailyMenuInputSchema)
def get_menu(date:str):
    """Useful when you need to know the menu for a specific date."""
    target_date= datetime.strptime(date, "%d/%m/%Y %H:%M") 
    weekday= target_date.strftime("%A")
    
    # Get the menu from the database
    return db.get_menu(day=weekday)

# Function to get the current date and time in the format dd/mm/yyyy hh:mm
@tool("current-datetime-tool")
def get_current_datetime():
    """Useful when you need to know the current date and time for date and time based operations."""        
    local_tz = pytz.timezone(local_timezone)
    return {
        "current_datetime": datetime.now(local_tz).strftime("%A, %d/%m/%Y %H:%M")
    } 
    
# We create an array containing the tools that will be used by the agent
tools=[operative_schedule, get_menu, get_current_datetime]
#endregion

As usual, The `system_message` has been updated with order management instructions.

Take some time to look at it, the way you give instructions to the LLM have a huge impact on its reasoning and is an area where you will spend most of your time doing prompt tuning.

In [12]:
# Updated system message, notice the presence of new task definitions

system_message = f'''
Role: An assistant with expertise in handling questions about operative schedule, available menu and ordering of a food delivery service.
 
Instructions:
- Provide information about the available menu, opening schedule and placing orders.
- Kindly deny any request not regarding opening schedule, available menu or order placement.
- Always assume the date and time returned by the get_current_datetime tool as the reference date for all date calculations.
- If a date is not indicated use the reference date.
- If a time is not indicated assume it is 12 PM.
- Assume the following format: 'DD/MM/YYYY HH:MM' as the standard format for dates.

Steps:
  - Opening Schedule:
      1. Always use the calculated date based on reference date to check and return the opening schedule.
      2. Always indicate the desired data and time including weekday in the response.
      
  - Available menu:
      1. When the user asks for the a menu you should respond using the following template enclosed in triple quotes:

      ```
      Menu for <day-name> <requested date>
      
      Pasta
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..      
      
      
      Pizza
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      
      Today's Special Pizzas
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..   
      
      Drinks
      
      1. <name> - <price> 
      2. <name> - <price> 
      3. ...
      4. .. 
      
      Dessert
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      ```  
      
      2. If the request is about specific menu entries (e.g. 'do you have pasta', 'do you offer vegetarian food' or 'are there any specials?' ), together with the menu kindle answer the question.     
      3. If no specials are available for the day, do not include that section into menu.
      4. If no menu is available, kindly reply that there you can serve any food for that day..
      5. If the request relates to an item that is not available in the menu, kindly reply that it cannot be ordered.      
  
  - Order Placement:    
      1. The user id is mandatory for placing an order. If the user id is available, kindly ask the user to provide it.
      2. The user must indicate when it wants to place the order. If the user does not provide a date and time, kindly ask the user to provide it.
      3. The ordering date must fall within the opening schedule. If the user tries to place an order outside the opening schedule, kindly inform the user that the service is closed at that time.
      4. The user must provide its the last 4 digits of registered credit card. If the user does not provide it, kindly ask the user to provide it, otherwise the order cannot be placed.
      5. When order is successfully placed, respond with the following message enclosed in triple quotes:
      
      ```
      Thank you for your order <first-name> <user_id>. 
            
      Your order <order-id> will be delivered on <date> at <time>.
      We will charge your credit card ending in <last-4-digits> for the amount of <total-amount>.      
      
      Your order details:
      <quantity>x <name> - <price>
      
      <thank the user depending if it is a new order or an update> <firstname> <lastname> <user_id>.
      ```
  
Expectation:
  - Provide a seamless experience to the user, by providing the requested information in a kind and timely manner, including all the necessary details and guiding the user to a possible follow up step.

Narrowing:  
  1. The user can only order food that is available in the menu. Any other request should be kindly denied.
  2. Deny all the requests referring to a date antecedent the reference date.
'''

It comes without surprise that we are now going to define another tool: **The one in charge of processing an order** 

Let's start by defining the tool signature.  

`date`: *is the order date*  
`user_id`: Is the optional *id of the user*, if present we'll use it to lookup customer info since we probably already have them.  
`credit_card_digits`: Are the *last 4 digits of the user credit card*, we use it as 'poor man' way to check its identity (ok, not super safe, but this is not going into production)  
`order_detail`: Is a string representing a json containing the *order* (see the code for its structure)  
`order_id`: The optional *id of the order*, when present it lets the user update it. 


In [13]:
# Define the input schema for the order tool
class OrderToolInputSchema(BaseModel):
    date: str = Field(description="The date and time for which you want the order to be delivered in format 'DD/MM/YYYY HH:MM' (e.g. '18/05/2024 11:23')")
    user_id: str = Field(description="The user id in form '#<4-digits-number>'.")    
    credit_card_digits: str = Field(description="The last 4 digits of the credit card number in form '####'")
    order_detail: str = Field(description="""The order information using the following JSON format:
                            [                                
                                {
                                    "kind": "<kind of food>", # example: "pasta,pizza,drink",
                                    "name": "<name of the food>", # example: "lasagna","coke"
                                    "quantity": <number of dishes> #  example: 2    
                                    "extras": ["<extra1>", "<extra2>", ...] # example: ["double cheese", "well cooked", "no ice"])
                                },
                                <next food>
                            ]
                            """)
    order_id: Optional[str] = Field(description="The id of the open order in form '<4-digits-number>' (e.g. 8642)")
    

And this is the tool who will handle the order.  
  
Don't focus too much on the code implementation, it just check that user is allowed to place/update an order, that ordered food is present in today's menu and then saves the order into local database.  

Once done it returns a json representing the `OrderResponse` below

```python
class OrderResponse(BaseModel):
    order_id: str = ""
    user_name: str = ""
    messages: List[str] = []
    status: str = "success"
    total_price: float = 0.0
    currency: str = "EUR"
    card_digits: str = ""
```

In [14]:

@tool("order-tool", args_schema=OrderToolInputSchema)
def order(date:str, user_id:str, order_detail: str, credit_card_digits:int,order_id:str=None,):
    """Useful when you need to process a wish from the user order for some food."""
    
    weekday=datetime.strptime(date, "%d/%m/%Y %H:%M").strftime("%A")
    total_price=0    
    response=OrderResponse()    
    
    # Check if the user id is valid, if not, return the info to the LLM
    user=db.get_customer(user_id)
    if user is None:
        response.messages.append(f"Sorry, I could not find any user with the user id: {user_id}. Please provide the id of a registered user.")
        response.status="failed"
        return response
    
    # Check if the provided card digits match the ones registered for the user
    if user.card_digits != credit_card_digits:
        response.messages.append(f"Sorry, the last 4 digits of the card provided do not match the ones registered for user {user_id}. Please provide the correct digits.")
        response.status="failed"
        return response
        
    # Get available dishes for the selected date  
    available_dishes=db.get_dishes(day=weekday)
    
    # Check if ordered dishes are present in the menu for the desired date
    detail=json.loads(order_detail)
    for detail in detail:
        dish_name=detail["name"]
        dish=available_dishes.get(dish_name.lower())
        if dish is None:
            response.messages.append(f"Sorry, {detail['name']} is not available in our menu for desired date.")
            response.status="failed"
        else:
            # Dish is fine, add price to the total price of the order 
            total_price+=dish.price*detail["quantity"]
    
    # If dish validation failed, return the response to the LLM        
    if response.status=="failed":
            return response
        
    # Validation passed, saves/updates the order into database.    
    order=Order(user_id=user_id, date=date, total=total_price, detail=order_detail, status="pending")
    order_id=db.upsert_order(order_id=order_id, order=order)
    
    # Return the order status for further processing by the LLM
    response.order_id=order_id
    response.user_name=user.first_name
    response.total_price=total_price
    response.card_digits=user.card_digits
        
    return response

Since in the first cell we defined the previous three tools, we must now add the new one we just created.

In [15]:
# We add the new tool to the list of tools available to the agent
tools.append(order)

Since we want now to start chatting with the agent, we need to keep track of the whole conversation, to do this, we use LangChain [ChatMessageHistory](https://js.langchain.com/v0.1/docs/modules/memory/chat_messages/).  
 
We'll use it to create an agent able to handle user conversations. 

In [16]:
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_id="0000"
memory = ChatMessageHistory(session_id=session_id)

This is the same code as the previous step, with the difference that it now uses a memory to keep track of the whole conversation with the user.  

See that the prompt has another [MessagePlaceHoler](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#messagesplaceholder) 'chat_history` where LangChain will add the user/agent conversation messages.


In [17]:
def run_agent(user_request:str):
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_message),  
        MessagesPlaceholder(variable_name="chat_history"),      # This is the chat history
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),  # This holds the agent interation messages
    ])

    # Create the agent
    agent = create_openai_functions_agent(llm, tools, prompt)
    # Create the runtime in charge of executing the agent (then one in charge of calling the tools, adding the returned data into message hsitory, call the llm again, etc.)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: memory,
        input_messages_key="input",
        history_messages_key="chat_history")
    
    # Execute the agent
    response= agent_with_chat_history.invoke({"input": user_request},config={"configurable": {"session_id": session_id}})
    return response["output"]

It's now time to chat with the agent and order some food 😎 

Run the cell and enter your request in the prompt box.

Keep in mind that now we have memory, so each message enteres will be a follow up of the previous one, like any real-chat application.

**Note:**  
Re-run all the cells above to start with a new session.

In [18]:
# Chat with the agent until the user types an empty message
while True:
    user_message = input("Enter your message: ")
    if user_message == '':
        break
    cprint(f"User: {user_message}","blue")
    try:
        result = run_agent(user_message)
        cprint(f"Agent: {result}", "yellow")
        await asyncio.sleep(0.5)
    except Exception as e:
        cprint(f"An error occurred: {str(e)}", "red")
    

User: Hi! What can I order today?


> Entering new AgentExecutor chain...

Invoking: `current-datetime-tool` with `{}`


{'current_datetime': 'Monday, 09/09/2024 11:57'}
Invoking: `daily-menu-tool` with `{'date': '09/09/2024 11:57'}`


day='Monday' pasta=[] pizza=[] specials=[] drinks=[] dessert=[]I'm sorry, but there is no available menu for today, Monday, 09/09/2024. 

Is there anything else I can assist you with?

> Finished chain.
Agent: I'm sorry, but there is no available menu for today, Monday, 09/09/2024. 

Is there anything else I can assist you with?
User: Can I order something tomorrow at noon?


> Entering new AgentExecutor chain...

Invoking: `schedule-tool` with `{'date': '10/09/2024 12:00'}`
responded: Let me first check our opening schedule for tomorrow, Tuesday, 10/09/2024 at 12:00 PM.

Please hold on a moment.

day='Tuesday' start='9:00' end='21:00' status='open'Yes, we are open tomorrow, Tuesday, 10/09/2024 from 9:00 AM to 9:00 PM.

Would you like to see the menu for

** Congratulations!** 🏆🏆🏆

You are now an Agent ninja! 🥷 and ready to build your own **AI Agent**.

Of course, its very simple, not very robust and not feature complete, but goal was to make it as simple as possible for better understanding.  

The next step [Notebook 08](08_final.ipynb) adds two more tools for searching your orders and eventually cancel them.  
  
If you still have energy, go and have a look! 🙂